In [25]:
from pathlib import Path
from src.visualization.render_mp4 import (
    mp4_from_pickle_jupyter,
    unpickle_data,
    pickle_data,
)
from src.common.files import get_raw_paths
import random
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt

In [27]:
rollout = get_raw_paths(
    environment="sim",
    demo_source="rollout",
    demo_outcome="success",
    task="round_table",
    randomness="low",
)

random.shuffle(rollout)

len(rollout), rollout[:3]

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/*.pkl*


(197,
 [PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T09:24:36.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T10:43:17.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T10:29:27.pkl')])

In [11]:
teleop = get_raw_paths(
    environment="sim",
    demo_source="teleop",
    demo_outcome="success",
    task="round_table",
    randomness="low",
)

random.shuffle(teleop)

len(teleop), teleop[:3]

Found the following paths:
    /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/teleop/low/success/*.pkl*


(50,
 [PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/teleop/low/success/2024-02-10T13:11:59.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/teleop/low/success/2024-02-13T20:29:29.pkl'),
  PosixPath('/data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/teleop/low/success/2024-02-13T19:56:52.pkl')])

In [45]:
from src.common.geometry import np_action_6d_to_quat
from scipy.spatial.transform import Rotation as R

In [51]:
for p in tqdm(rollout):
    path = Path(p)
    try:
        data = unpickle_data(path)

        # Print some stats on the action to see if it's likely pos or delta actions
        actions = np.array(data["actions"])
        ma = np.max(np.abs(actions[:, :3]), axis=0)
        robot_states = np.stack([o["robot_state"] for o in data["observations"]])

        # If any of the actions are bigger than 0.1 (10 cm), it's likely pos actions
        if np.any(ma > (0.1)):
            print(f"Teleop {path} likely pos actions", ma.max())
            data["action_type"] = "pos"

            assert actions.shape[1] == 10
            # IF we've used rot_6d convert to quat
            actions = np_action_6d_to_quat(actions)
            assert actions.shape[1] == 8

            # Get the action quat
            pos_action_quat = R.from_quat(actions[:, 3:-1])

            # Get the position quat from the robot state
            pos_quat = R.from_quat(robot_states[:-1, 3:7])

            # The action quat was calculated as pos_quat * action_quat
            # Calculate the delta quat between the pos_quat and the action_quat
            delta_action_quat = pos_quat.inv() * pos_action_quat

            # Also calculate the delta position
            delta_action_pos = actions[:, :3] - robot_states[:-1, :3]

            # Insert the delta quat into the actions
            actions = np.concatenate(
                [delta_action_pos, delta_action_quat.as_quat(), actions[:, -1:]], axis=1
            )

            data["actions"] = actions.tolist()

        else:
            print(f"Teleop {path} likely delta actions")
            data["action_type"] = "delta"

        pickle_data(data, path)
    except EOFError:
        print(f"Failed to load {path}")

plt.show()

  0%|          | 0/197 [00:00<?, ?it/s]

Teleop /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T09:24:36.pkl likely pos actions 0.6139788627624512


  1%|          | 1/197 [00:04<13:33,  4.15s/it]

Teleop /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T10:43:17.pkl likely pos actions 0.6019027233123779


  1%|          | 2/197 [00:21<38:11, 11.75s/it]

...

 99%|█████████▉| 195/197 [36:33<00:22, 11.09s/it]

Teleop /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T10:36:22.pkl likely pos actions 0.5926955938339233


 99%|█████████▉| 196/197 [36:41<00:10, 10.27s/it]

Teleop /data/scratch-oc40/pulkitag/ankile/furniture-data/raw/sim/round_table/rollout/low/success/2024-02-19T09:20:50.pkl likely pos actions 0.6050562858581543


100%|██████████| 197/197 [36:51<00:00, 11.23s/it]


In [42]:
data = unpickle_data(rollout[0])

actions = np.array(data["actions"])
robot_states = np.stack([o["robot_state"] for o in data["observations"]])

In [43]:
actions.shape, robot_states.shape

((905, 10), (906, 14))

In [44]:
assert actions.shape[1] == 10
# IF we've used rot_6d convert to quat
actions = np_action_6d_to_quat(actions)
assert actions.shape[1] == 8

# Get the action quat
pos_action_quat = R.from_quat(actions[:, 3:-1])

# Get the position quat from the robot state
pos_quat = R.from_quat(robot_states[:, 3:7])

# The action quat was calculated as pos_quat * action_quat
# Calculate the delta quat between the pos_quat and the action_quat
delta_action_quat = pos_quat.inv() * pos_action_quat

# Insert the delta quat into the actions
actions = np.concatenate(
    [actions[:, :3], delta_action_quat.as_quat(), actions[:, -1:]], axis=1
)

In [46]:
actions.tolist()

[[0.567752480506897,
  0.045227229595184326,
  0.1193181648850441,
  -0.8911190777151187,
  -0.4536913313206228,
  0.006570408799938945,
  0.005272091129610978,
  -1.0],
 [0.570074737071991,
  0.038280144333839417,
  0.11789141595363617,
  -0.8918748337953879,
  -0.4521793830584743,
  0.006225127746231825,
  0.007371171131496801,
  -0.9986537098884583],
 [0.5690387487411499,
  0.03159600496292114,
  0.11618205159902573,
  -0.8927475567460248,
  -0.4504051769174174,
  0.00658771859119615,
  0.009673597766927085,
  -0.9963245987892151],
 [0.5654342770576477,
  0.026742011308670044,
  0.11337225139141083,
  -0.8933463007241632,
  -0.44910007839282345,
  0.008905434273120095,
  0.012735768937132866,
  -0.9955458045005798],
 [0.564630389213562,
  0.02093413472175598,
  0.11134663224220276,
  -0.8949625170071235,
  -0.4459565504850084,
  0.0053755767258695355,
  0.01165982018088842,
  -0.9973685145378113],
 [0.5626800060272217,
  0.014659717679023743,
  0.10923335701227188,
  -0.896602222440

## Debug combining datasets

In [4]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [5]:
from src.dataset.zarr import combine_zarr_datasets
from src.common.files import get_processed_paths

In [6]:
paths = get_processed_paths(
    environment="sim",
    demo_source="teleop",
    demo_outcome="success",
    task=["round_table", "lamp"],
    randomness="low",
)

Found the following paths:
    /data/scratch/ankile/furniture-data/processed/sim/round_table/teleop/low/success.zarr
    /data/scratch/ankile/furniture-data/processed/sim/lamp/teleop/low/success.zarr


In [8]:
combine_zarr_datasets(
    zarr_paths=paths[1:],
    max_episodes=None,
    keys=[
        "color_image1",
        "color_image2",
        "robot_state",
        "action/pos",
        "skill",
    ],
)

Loading zarr files:   0%|          | 0/1 [00:00<?, ?it/s]


































































































Loading zarr files: 100%|██████████| 1/1 [00:47<00:00, 47.68s/it]


{'episode_ends': array([  786,  1699,  2540,  3419,  4177,  4928,  5819,  6665,  7473,
         8377,  9269, 10056, 10862, 11750, 12573, 13403, 14532, 15474,
        16310, 17239, 17953, 18684, 19738, 20534, 21279, 22203, 22952,
        23815, 24861, 25709, 26639, 27421, 28182, 28969, 29894, 30696,
        31627, 32673, 33823, 34604, 35330, 36179, 37182, 37996, 38912,
        39807, 40706, 41578, 42402, 43267]),
 'furniture': ['lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp',
  'lamp'],
 'success': array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 